In [6]:
import numpy as np
import pandas as pd 
pd.core.common.is_list_like = pd.api.types.is_list_like 
import pandas_datareader as web 
from pandas_datareader import data as pdr 
import yfinance as yf 
yf.pdr_override()  
import datetime  
import matplotlib.pyplot as plt 
 
import warnings 
warnings.filterwarnings("ignore") 

# function to get the price data from yahoo finance  
def getDataBatch(tickers, startdate, enddate): 
    def getData(ticker): 
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate)) 
    datas = map(getData, tickers)    
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

In [24]:
# 1
# Define the start and end dates
start_date = datetime.datetime(2008, 12, 31)
end_date = datetime.datetime(2018, 12, 31)

# Define the list of tickers
tickers = ['AAPL', 'AMZN', 'GS']

# Download the data using pandas-datareader
stock_data = getDataBatch(tickers, start_date, end_date)
daily_close_px = stock_data.reset_index().pivot(index='Date', columns='Ticker', values='Adj Close') 

# Calculate the daily percentage change for `daily_close_px` 
ret_daily = daily_close_px.pct_change().dropna() 

# add the portfolio returns 
wts_port = np.ones([3,1])/3 
ret_stocks = ret_daily.loc[:,['AAPL', 'AMZN', 'GS']] 
ret_daily['Port'] = np.dot(ret_stocks,wts_port) 

# Calculate the annualized geometric mean return 
annual_stock_returns = (1 + ret_daily.mean())**252 - 1
print("Annualized geometric mean returns:")
print(annual_stock_returns)

#calculate annualized mean  
Ret_mean_Annual = ret_daily.mean()*252 
print("Annualized simple mean returns:")
print(Ret_mean_Annual)
#calculate annualized standard deviation 
Ret_std_Annual = ret_daily.std()*np.sqrt(252)
print("Annualized simple standard deviation:")
print(Ret_std_Annual)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Annualized geometric mean returns:
Ticker
AAPL    0.354567
AMZN    0.484238
GS      0.137258
Port    0.317442
dtype: float64
Annualized simple mean returns:
Ticker
AAPL    0.303665
AMZN    0.395211
GS      0.128653
Port    0.275843
dtype: float64
Annualized simple standard deviation:
Ticker
AAPL    0.266648
AMZN    0.344107
GS      0.315085
Port    0.237586
dtype: float64


In [42]:
#2

port1day = 100*(ret_daily['Port']+1)
# find historical var using direct percentile on portfolio values
#    report the oposite of the loss
var05Hist = 100-np.percentile(port1day,5)
print("5% VaR Historical data {0:8.4f}".format(var05Hist) )

# find historical var using direct percentile on portfolio values
var01Hist = 100-np.percentile(port1day,1)
print("1% VaR Historical data {0:8.4f}".format(var01Hist) )

5% VaR Historical data   2.3001
1% VaR Historical data   4.1493


In [41]:
#3

s = np.std(ret_daily['Port'])
m = np.mean(ret_daily['Port'])

# VaR(0.05) 5% percentile  
RStar05 = stats.norm.ppf(0.05, m, s)
var05Norm = -100*RStar05
#print("5% VaR Normal"+ "%8.2f" % var05Norm)
print("5% VaR Normal {0:8.4f}".format(var05Norm))

# Normal Analytical VaR(0.01) 
RStar01 = stats.norm.ppf(0.01, m, s)
var01Norm = -100*RStar01
print("1% VaR Normal {0:8.4f}".format(var01Norm))

5% VaR Normal   2.3518
1% VaR Normal   3.3716


In [40]:
#4

# fit the return time series using a student-t distribution
tdf, tmean, tsigma = stats.t.fit(ret_daily['Port'])

p1 = 0.01 # VaR confidence level
p2 = 0.05
# --- Use Student-t Inverse cdf function to calculate analytical VaR

R_star_t = stats.t.ppf(p2, tdf, tmean, tsigma)
VaR_Rstar_t =  - 100*R_star_t
print("5% student-t Analytical VaR {0:8.4f}".format(VaR_Rstar_t))
R_star_t1 = stats.t.ppf(p1, tdf, tmean, tsigma)
VaR_Rstar_t1 =  - 100*R_star_t1
print("1% student-t Analytical VaR {0:8.4f}".format(VaR_Rstar_t1))

5% student-t Analytical VaR   2.1344
1% student-t Analytical VaR   4.0798


In [1]:
#5


N = 10000; #number of simulations

np.random.seed(1)

# generate randrom returns using fitted student-t parameters from above,
PortRet_MC_t = stats.t.rvs(tdf, loc=tmean, scale=tsigma, size=N)
# Make sure loss doesn't exceed -100%
PortRet_MC_t[PortRet_MC_t<-1.0]=-1.0;

PortRet_MC_t_Sorted= np.sort(PortRet_MC_t)

ID_At_p = round(p2*N)-1

PortRet_At_p_MC_t = PortRet_MC_t_Sorted[ID_At_p]
PortRet_MC_LeftTail_t = PortRet_MC_t_Sorted[0:(ID_At_p+1)]

VAR_At_p_MC_t = - 100* PortRet_At_p_MC_t

print("5% student-t Monte Carlo VaR {0:8.4f}".format(VAR_At_p_MC_t))

ID_At_p1 = round(p1*N)-1

PortRet_At_p_MC_t1 = PortRet_MC_t_Sorted[ID_At_p1]
PortRet_MC_LeftTail_t1 = PortRet_MC_t_Sorted[0:(ID_At_p1+1)]

VAR_At_p_MC_t1 = - 100* PortRet_At_p_MC_t1

print("1% student-t Monte Carlo VaR {0:8.4f}".format(VAR_At_p_MC_t1))

NameError: name 'stats' is not defined